In [1]:
from utils.util import *
import os
import random

CSV_FILE_PATH = '../dataset/image_labels_with_code.csv'  # Replace with your CSV file path
IMAGE_DIR = '../data/InsPLAD-fault/defect_supervised'  # Replace with the base directory of your images

IMG_HEIGHT = 224  # Image height for ResNet
IMG_WIDTH = 224  # Image width for ResNet
BATCH_SIZE = 32
NUM_WORKERS = 4  # Number of worker processes for DataLoader
LEARNING_RATE = 1e-4
NUM_EPOCHS = 50  # Max number of epochs (early stopping will be used)
EARLY_STOPPING_PATIENCE = 5
BEST_MODEL_PATH = '../models/defect_detection_lighting_model.pth'
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

# Glass insulator


In [2]:
df = load_data_from_csv(CSV_FILE_PATH)
train_df, val_df = split_data(df, stratify_col='status')  # Ensure 'status' is the correct column
df = df[df['category_code']==1]

    # Define image transformations
    # For ResNet, normalization values are typically from ImageNet
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
        transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        # Add more augmentations if needed (e.g., ColorJitter)
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ])

val_transform = transforms.Compose([
        transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ])

train_loader, val_loader = get_data_loaders(
        train_df, val_df, IMAGE_DIR, train_transform, val_transform, BATCH_SIZE, NUM_WORKERS
    )
model = get_pretrained_resnet(num_classes=1, pretrained=False)
model.load_state_dict(torch.load(BEST_MODEL_PATH))
model.to(DEVICE)
model.eval()
criterion = nn.BCEWithLogitsLoss()


Successfully loaded CSV from: ../dataset/image_labels_with_code.csv
CSV columns: ['category_code', 'category', 'image_path', 'status']
Data split: 10372 training samples, 1153 validation samples.
Stratified by column: 'status'
Training status distribution:
status
0    0.755688
1    0.211917
2    0.032395
Name: proportion, dtype: float64
Validation status distribution:
status
0    0.755421
1    0.212489
2    0.032090
Name: proportion, dtype: float64
DataLoaders created. Training batches: 325, Validation batches: 37
ResNet18 loaded. Final layer replaced for 1 output features.
Only the final layer will be trained initially.


In [3]:
evaluate_model(model, val_loader, criterion, DEVICE)


(0.7360395635611475,
 0.4163052905464007,
 0.22163588390208924,
 0.5266457680085691,
 0.31197771170238875)

In [4]:
def predict_single(img_path, model, device):
    img_tfms = transforms.Compose([
        transforms.Resize((224, 224)),  # <— same size you used
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],  # <— same stats you used
                             [0.229, 0.224, 0.225]),
    ])

    """Return probability and binary label (0=good, 1=defect) for one image."""
    model.eval()                                       # inference mode
    img = Image.open(img_path).convert('RGB')
    x   = img_tfms(img).unsqueeze(0).to(device)        # shape 1×3×H×W

    with torch.no_grad():
        logit = model(x)                               # shape [1, 1]
        prob  = torch.sigmoid(logit).item()            # 0 – 1
        label = int(prob > 0.5)                        # threshold

    return prob, label

In [6]:
num_test_images = 5
image_dir = "../data/InsPLAD-fault/unsupervised_anomaly_detection/lightning-rod-suspension/test/rust"
#image_dir = "..//data/InsPLAD-fault/defect_supervised/glass-insulator/val"

# Get a list of all image files
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

for _ in range(num_test_images):
    img_path = random.choice(image_files)
    print(f"\n--- Processing image: {os.path.basename(img_path)} ---")

    prob, label = predict_single(img_path, model, DEVICE)
    print(f'File: {img_path}')
    print(f'Defect probability: {prob:.2%}')
    print('Predicted label   :', 'defect' if label else 'good')


--- Processing image: 15-06-2021_DJI_0149_742.jpg ---
File: ../data/InsPLAD-fault/unsupervised_anomaly_detection/lightning-rod-suspension/test/rust/15-06-2021_DJI_0149_742.jpg
Defect probability: 49.79%
Predicted label   : good

--- Processing image: 09-06-2021_DJI_0058_456.jpg ---
File: ../data/InsPLAD-fault/unsupervised_anomaly_detection/lightning-rod-suspension/test/rust/09-06-2021_DJI_0058_456.jpg
Defect probability: 39.50%
Predicted label   : good

--- Processing image: 09-06-2021_DJI_0299_526.jpg ---
File: ../data/InsPLAD-fault/unsupervised_anomaly_detection/lightning-rod-suspension/test/rust/09-06-2021_DJI_0299_526.jpg
Defect probability: 55.35%
Predicted label   : defect

--- Processing image: 02-06-2021_DJI_0045_193.jpg ---
File: ../data/InsPLAD-fault/unsupervised_anomaly_detection/lightning-rod-suspension/test/rust/02-06-2021_DJI_0045_193.jpg
Defect probability: 60.53%
Predicted label   : defect

--- Processing image: 15-06-2021_DJI_0050_728.jpg ---
File: ../data/InsPLAD-fau